## Un emjemplo imaginario
Imaginemos que tenemos un cliente que quiere que se procesen 501 peticiones al mismo tiempo sobre diversa información de stock.  Las peticiones de información se procesan en nuestras máquinas utilizando un recurso limitado llamado procesador.  
- El procesador puede procesar 10 peticiones al mismo tiempo. No podemos tener mitad de procesador
- Cada máquina tiene 5 procesadores.  No podemos tener mitad de maquina
- Si la solicitud de información se realiza después de las 12:00, se considera hora punta y es posible que la peticion compita por los recursos.  Por eso, el cliente necesitará una asignación adicional del 25%.

Esta pregunta requiere varias conversiones de unidades, de petición a procesadores y a máquinas.
Las variables son las peticiones y la hora del dia.


In [1]:
peticiones = 501

In [3]:
def convert_request_to_processors(peticiones):
    return peticiones / 10

In [4]:
convert_request_to_processors(peticiones)

50.1

No podemos tener la mitad de un procesador en este ejemplo.  Tenemos que redondear

In [5]:
import math

In [6]:
def convert_request_to_processors(request):
    return math.ceil(request/10)

In [8]:
convert_request_to_processors(peticiones)

51

Ahora debemos convertir los procesadores en máquinas

In [9]:
def convert_processors_to_machines(processors):
    return math.ceil(processors / 5)

In [10]:
convert_processors_to_machines(51)

11

Si la peticiones se realiza antes de las 12:00, necesitaremos 11 máquinas para procesar 501 peticiones al mismo tiempo.  Si la peticiones se realiza después de las 12.00, debemos tener en cuenta que se necesita un 25% más de capacidad

In [11]:
import datetime

In [12]:
def apply_additional_capacity(machines, time):
    if time >= datetime.time(12, 0):
        additional_capacity = math.ceil(machines * .25)
        machines += additional_capacity
    return machines

In [13]:
apply_additional_capacity(11, datetime.time(12,1))

14

Ahora vamos a crear la interfaz de usuario para estos cálculos básicos, utilizando el panel https://panel.holoviz.org/  

In [16]:
import panel as pn
pn.extension()

¿Cuáles son nuestras entradas? Peticiones y hora del día. 

In [19]:
## request_input = pn.widgets.IntInput(name='Requests', value=1, step=1, start=1, end=1000000)
time_input = pn.widgets.DatetimeInput(name='Time of request', value=datetime.datetime(2019, 2, 8, 0,0), format ='%H:%M') # requires datetime package
app = pn.Row(request_input,time_input)
app

Row
    [0] IntInput(end=1000000, name='Requests', start=1, value=14)
    [1] DatetimeInput(format='%H:%M', name='Time of request', type=<class 'datetime.datetime'..., value=datetime.datetime(2019, ...)

Ahora querremos una función que procese nuestro cálculo, un botón que dispare esa función y una salida para el cálculo

In [20]:
machines_output = pn.widgets.IntInput(name='Machines', value = 0, disabled= True)

In [21]:
def on_click_calculate(event):
    processors = convert_request_to_processors(request_input.value)
    machines = convert_processors_to_machines(processors)
    if time_input.value.time() >= datetime.time(12, 0):
        additional_capacity = peak_time_capacity(machines)
        machines += additional_capacity
    machines_output.value = machines

In [22]:
calculate_button = pn.widgets.Button(name='Calculate', button_type='primary')
calculate_button.on_click(on_click_calculate)

Watcher(inst=Button(button_type='primary', name='Calculate'), cls=<class 'panel.widgets.button.Button'>, fn=<function on_click_calculate at 0x7fa2cd573310>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [23]:
app = pn.Column(pn.Row(request_input,time_input), calculate_button, machines_output)

**NOTA:** He tenido problemas con el botón ui en jupyter lab, si te encuentras con esto también ejecutar en jupyter classic. <br>
Hice esto yendo a  Help > Launch Classic Notebook <br>
![image.png](images/launch_classic.png)

In [ ]:
app.servable()

En este punto tenemos esencialmente el esqueleto de nuestra calculadora, conocemos nuestras entradas y salidas y hemos definido funciones para la lógica de la calculadora